In [0]:
%python
%run "/Workspace/Users/ravipalepu33@gmail.com/Vehicle-Insurance/Branch_data"

In [0]:
claims_data_df = spark.read.option("header", "true").format("csv").load("/FileStore/tables/claims_data.csv")

In [0]:
display(claims_data_df.limit(10))

In [0]:
from pyspark.sql.functions import rand

# Get unique branch IDs from branch data
branch_ids = branch_data_df.select("Branch_ID").distinct().rdd.flatMap(lambda x: x).collect()

# Define a UDF to randomly assign a Branch_ID
import random
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

random_branch_udf = udf(lambda: random.choice(branch_ids), StringType())

# Add the Branch_ID column to claims data
claims_data_df = claims_data_df.withColumn("Branch_ID", random_branch_udf())

# Show results
claims_data_df.show()


In [0]:
claims_data_df.write.mode("overwrite").saveAsTable("claims_data")

In [0]:
%sql
SELECT * FROM claims_data LIMIT 10

In [0]:
%python
from pyspark.sql.functions import col, sum, when

null_cnt = claims_data_df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in claims_data_df.columns])
display(null_cnt.limit(10))

In [0]:
%python
claims_data_df.filter(
    (claims_data_df.Branch_ID.isNotNull()) & 
    (claims_data_df.Policy_ID.isNotNull())
).groupBy(
    "Branch_ID", "Status", "Vehicle_ID", "Claim_Amount", "Policy_ID"
).count().filter(
    "count > 1"
).show()

In [0]:
display(claims_data_df.limit(10))

In [0]:
%sql
SELECT Branch_ID,COUNT(Policy_ID) AS cnt FROM claims_data GROUP BY Branch_ID ORDER BY cnt DESC LIMIT 10 

In [0]:
%sql
SELECT 
Branch_ID,AVG(Claim_Amount) AS avg_claim_amount 
FROM claims_data 
GROUP BY Branch_ID 
ORDER BY avg_claim_amount 
DESC LIMIT (10)

In [0]:
%sql
SELECT Branch_ID,COUNT(Status) AS cnt FROM claims_data WHERE Status = 'Approved' GROUP BY Branch_ID ORDER BY cnt DESC LIMIT 10

In [0]:
%sql
SELECT Policy_ID,COUNT(Claim_ID) AS cnt FROM claims_data GROUP BY Policy_ID ORDER BY cnt DESC LIMIT 10

In [0]:
%sql
SELECT Branch_ID,COUNT(Claim_ID) AS cnt FROM claims_data GROUP BY Branch_ID ORDER BY cnt DESC LIMIT 10

In [0]:
%sql
SELECT * FROM branch_data LIMIT 10

In [0]:
%sql
SELECT b.Location,COUNT(c.Policy_ID) AS policy_count_per_location 
FROM Branch_data b 
INNER JOIN Claims_data  c on b.Branch_ID = c.Branch_ID 
WHERE Location IS NOT NULL 
GROUP BY b.Location
ORDER BY policy_count_per_location 
DESC LIMIT 10